In [1]:
import utils

/root/miniconda3/envs/ThesisRobot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path2data = "../blendshape"

SKIP_FRAMES = 1

ids, labels, groups, catgs = utils.get_vids(path2data)
utils.split_dataset(ids, labels, groups, skipframes=SKIP_FRAMES)

Name		 ids labels groups
--------------------------------------
Train: 		 91 91 91
Validation: 	 23 23 23
--------------------------------------
Name distribution:	 YES - NO
Original distribution:	 0.3333333333333333 - 0.6666666666666666
Train distribution:	 0.32967032967032966 - 0.6703296703296703
Val distribution:	 0.34782608695652173 - 0.6521739130434783
--------------------------------------
Succesfully saved at ../pickled/ with skipframes = 1 []


In [3]:
SKIP_FRAMES = 1
train_ds, val_ds = utils.get_dataset(skipframes=SKIP_FRAMES)

Succesfully retrieved at ../pickled/ with skipframes = 1 []


In [4]:
train_ds[0][0].shape

torch.Size([40, 52])